In [280]:
# Import relevant packaages
import ee
import geemap

In [281]:
# Initialisie earth engine
ee.Authenticate()
ee.Initialize()

Enter verification code: 4/1AX4XfWgkuqU7K6ZUamUnO6rbfqPYgzsIvw3NHgzIX6UkEIgAmZTTh4mKQpI

Successfully saved authorization token.


In [282]:
# magic command to render plots in notebook
%matplotlib inline

### Broadleaf cover

In [444]:
# Load a polygon of the UK so that we can clip the CORINE dataset
ukPoly = ee.FeatureCollection('USDOS/LSIB_SIMPLE/2017') \
         .filter(ee.Filter.eq('country_co', 'UK'))

# Load the Corine landcover categorisation for 2018 (the latest)
# Each pixel represents 100m x 100m or 1 hectare
# Select the landcover band and clip to the UK border
corine = ee.Image('COPERNICUS/CORINE/V20/100m/2018') \
    .select('landcover') \
    .clipToCollection(ukPoly)

# Select pixels that represent Broadleaf categorisation (311)
mask = corine.eq(311)
# Generate a broadleaf image by using the mask for 311
broadleaf = mask.updateMask(mask).multiply(corine)

In [445]:
# Create a map
Map = geemap.Map()
# Centre on the UK
Map.setCenter(-1.2589, 51.7521, 8)
# Plot the map
Map.addLayer(broadleaf, {'palette' : 'red'}, 'Broadleaf', True)

If we load the administrative boundariies for the Uk, we can plot them and then calculate the percentage of landcover within a county that is broadleaf woodland.

In [446]:
# Select UK Counties from the GAUL feature collection
# "U.K. of Great Britain and Northern Ireland"

uk = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level2") \
              .filter(ee.Filter.eq('ADM0_NAME', "U.K. of Great Britain and Northern Ireland") \
)

Let's visualise the broadleaf woodland and county boundaries

In [447]:
# Plot map
Map.addLayer(uk, {}, 'uk counties')
Map

Map(center=[51.7521, -1.2589], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chi…

What is the total area of broadleaf woodland within the UK? Let's start with the total area of the UK

In [448]:
# disolve feature collection into single polygon
# then extract area
# convert from m2 to km2
area = uk.geometry().area().getInfo()/1e6
print("Total uk land area: ", area, " km2")

Total uk land area:  243266.3252841075  km2


Then we can work out the total area covered by broadleaf woodland

In [449]:
areaImage = mask.multiply(ee.Image.pixelArea())
area = areaImage.reduceRegion(
    reducer   = ee.Reducer.sum(),
    geometry  = uk.geometry(),
    scale     = 100,
    maxPixels = 1e10
    )

In [450]:
broadleafCover = area.getInfo()

What is the total area of broadleaf trees iin the UK?

In [451]:
print("Total uk broadleaf forest area: ", broadleafCover["landcover"] / (1e6), " km2")

Total uk broadleaf forest area:  5454.368235294117  km2


### Broadleaf woodland area by UK county

Not let's calculate the percentage broadleaf cover for each UK county, and then order them

In [452]:
# Define broadleaf mask for UK
ukBroadleafImage = mask.multiply(ee.Image.pixelArea())

# Functionn to map over counties (features) within UK featureCollection
def mapCounties(county):
    
    # Calculate county area and set property
    countyArea = county.geometry().area().divide(1e6).round()
    # Set a property within the county feature to capture the county area
    county = county.set('County_Area', countyArea)
    
    # Calculate broadleaf area within county
    broadleafArea = ukBroadleafImage.reduceRegion(
        reducer   = ee.Reducer.sum(),
        geometry  = county.geometry(),
        scale     = 100,
        maxPixels = 1e10
    ).get("landcover")
    
    # Convert broadleaf area into km2
    broadleafArea = ee.Number.float(broadleafArea).divide(1e6)
    
    # Set a property within the county feature to capture broadleaf area for the county
    county = county.set('Broadleaf_Area', broadleafArea)
    
    # Percentage broadleaf
    percentageBroadleaf = broadleafArea.divide(countyArea).multiply(100)
    # Set a property within the county feature to capture the % broadleaf
    county = county.set('Broadleaf_Per', percentageBroadleaf)

    # return updated feature
    return county
    
# Map over uk counties and calulcate the county area and the broadleaf woodland area within it
broadleafAreaByCounty = uk.map(mapCounties)

In [453]:
broadleafAreaByCounty.aggregate_sum("Broadleaf_Area").getInfo()

5454.317529411764

In [454]:
broadleafByCounty_df = geemap.ee_to_df(broadleafAreaByCounty)

In [455]:
broadleafByCounty_df[["ADM2_NAME","County_Area","Broadleaf_Area","Broadleaf_Per"]] \
.sort_values("Broadleaf_Per", ascending=False) \
.head(10)

,ADM2_NAME,County_Area,Broadleaf_Area,Broadleaf_Per
39,Surrey,1677,185.679176,11.072104
43,West Sussex,1940,173.617373,8.949349
18,Hampshire,3738,280.165137,7.495055
13,East Sussex,1742,115.605608,6.636372
2,Berkshire,1219,79.154510,6.493397
23,Kent,3694,236.598902,6.404951
92,West Glamorgan,818,47.270863,5.778834
3,Buckinghamshire,1901,100.002235,5.260507
15,Gloucestershire,2619,130.236824,4.972769
22,Isle of Wight,374,17.354902,4.640348


Plot of percentage of broadleaf coverage across the UK

In [457]:
empty = ee.Image().byte()

# Paint all the county polygon edges with the same number and width, display.
outline = empty.paint(
  featureCollection = broadleafAreaByCounty,
  color = 'Broadleaf_Per'
)
visParams = {
  'palette': ['white', 'green'],
  'min': 0,
  'max': 11,
  'opacity': 1,
}

Map.addLayer(outline, visParams, 'broadleaf coverage')
Map

Map(bottom=5751.0, center=[51.467696956223385, -1.8896484375000002], controls=(WidgetControl(options=['positio…

### Broadleaf connectivity analysis